In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from catboost import CatBoostClassifier
import xgboost as xgb
import random

In [3]:
data = pd.read_csv("minute_data/BTC-USD_1M_SIGNALS.csv",)

In [4]:
data

,Unix,Date,Symbol,Open,High,Low,Close,Volume USD,Variation,RSI,MACD,MACD_H,-DM,+DM,ADX14
0,1483228860,2017-01-01 00:01:00,BTC/USD,966.34,966.34,966.34,966.34,-0.312459,-0.006352,-0.291721,0.003268,0.001095,-0.296745,-0.163178,-0.618641
1,1483228920,2017-01-01 00:02:00,BTC/USD,966.16,966.37,966.16,966.37,-0.310186,-0.000855,0.828292,0.003534,0.002246,-0.285351,-0.163178,-0.668323
2,1483228980,2017-01-01 00:03:00,BTC/USD,966.37,966.37,966.37,966.37,-0.348678,-0.000855,0.828292,0.003738,0.003484,-0.296745,-0.163178,-0.647506
3,1483229040,2017-01-01 00:04:00,BTC/USD,966.37,966.37,966.37,966.37,-0.348678,-0.011849,0.835669,0.003932,0.004996,-0.293490,-0.163178,-0.649919
4,1483229100,2017-01-01 00:05:00,BTC/USD,966.43,966.43,966.43,966.43,-0.348169,-0.028336,0.910591,0.004110,0.006831,-0.288606,-0.163178,-0.652332
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2674991,1646105880,2022-03-01 03:38:00,BTC/USD,43077.82,43077.82,43049.46,43049.46,-0.343969,0.205143,3.243746,202.140130,315.950658,-0.296745,1.189788,0.570316
2674992,1646105940,2022-03-01 03:39:00,BTC/USD,43035.16,43035.16,42999.44,42999.44,-0.174815,-0.069151,3.243036,182.098937,328.738652,0.603408,-0.163178,0.176675
2674993,1646106000,2022-03-01 03:40:00,BTC/USD,43022.24,43022.24,43016.03,43016.03,-0.346723,-0.126531,3.243035,153.986631,318.063305,-0.177376,-0.163178,-0.189747
2674994,1646106060,2022-03-01 03:41:00,BTC/USD,43018.23,43046.59,43018.23,43046.58,-0.318365,-0.000855,3.243946,115.826724,271.530088,1.241492,-0.163178,-0.539146


In [39]:
data[0:20]

,Unix,Date,Symbol,Open,High,Low,Close,Volume USD,Variation,RSI,MACD,MACD_H,-DM,+DM,ADX14
0,1483228860,2017-01-01 00:01:00,BTC/USD,966.34,966.34,966.34,966.34,-0.312459,-0.006352,-0.291721,0.003268,0.001095,-0.296745,-0.163178,-0.618641
1,1483228920,2017-01-01 00:02:00,BTC/USD,966.16,966.37,966.16,966.37,-0.310186,-0.000855,0.828292,0.003534,0.002246,-0.285351,-0.163178,-0.668323
2,1483228980,2017-01-01 00:03:00,BTC/USD,966.37,966.37,966.37,966.37,-0.348678,-0.000855,0.828292,0.003738,0.003484,-0.296745,-0.163178,-0.647506
3,1483229040,2017-01-01 00:04:00,BTC/USD,966.37,966.37,966.37,966.37,-0.348678,-0.011849,0.835669,0.003932,0.004996,-0.293490,-0.163178,-0.649919
4,1483229100,2017-01-01 00:05:00,BTC/USD,966.43,966.43,966.43,966.43,-0.348169,-0.028336,0.910591,0.004110,0.006831,-0.288606,-0.163178,-0.652332
5,1483229160,2017-01-01 00:06:00,BTC/USD,966.58,966.58,966.58,966.58,-0.347099,-0.000855,1.110668,0.004124,0.008586,-0.296745,-0.163178,-0.665621
6,1483229220,2017-01-01 00:07:00,BTC/USD,966.58,966.58,966.58,966.58,-0.348678,-0.000855,1.110668,0.003720,0.009398,-0.296745,-0.163178,-0.721027
7,1483229280,2017-01-01 00:08:00,BTC/USD,966.58,966.58,966.58,966.58,-0.348678,-0.000855,0.063198,0.003137,0.009823,-0.296745,-0.163178,-0.740746
8,1483229340,2017-01-01 00:09:00,BTC/USD,966.58,966.58,966.58,966.58,-0.348678,0.000977,0.063198,0.002335,0.009628,-0.296745,-0.162861,-0.762462
9,1483229400,2017-01-01 00:10:00,BTC/USD,966.57,966.57,966.57,966.57,-0.345367,0.022965,0.987989,0.001263,0.008497,-0.296745,-0.163178,-0.786936


In [5]:
predict_data = data.copy().drop(["Open","Close","High","Low", "Symbol"],axis=1)
max_days = 5
target_range = 2
for i in range(1,max_days):#2jours
    #predict_data[["Variation-{}".format(i),"Vol-{}".format(i),"RSI-{}".format(i),"MACD-{}".format(i),"MACD_H-{}".format(i),"CONF-{}".format(i),"TRANS-{}".format(i),"REV-{}".format(i),"FnG-{}".format(i)]] = data[["Variation","Volume","RSI","MACD","MACD_H","Confirmation Time","Transactions","Miners Revenue","FnG"]].shift(i)
    #predict_data[["Variation-{}".format(i),"Vol-{}".format(i),"RSI-{}".format(i),"MACD-{}".format(i),"MACD_H-{}".format(i),"CONF-{}".format(i),"TRANS-{}".format(i),"REV-{}".format(i),"FnG-{}".format(i), "ADX-{}".format(i), "+DM-{}".format(i), "-DM-{}".format(i)]] = data[["Variation","Volume","RSI","MACD","MACD_H","Confirmation Time","Transactions","Miners Revenue","FnG","ADX14","+DM","-DM"]].shift(i)
    predict_data[["Variation-{}".format(i),"Vol-{}".format(i),"RSI-{}".format(i),"MACD-{}".format(i),"MACD_H-{}".format(i)]] = data[["Variation","Volume USD","RSI","MACD","MACD_H"]].shift(i)
#predict_data["Target"] = (data["Variation"].shift(-1) >= 0)
predict_data["Target"] = (data["Close"].shift(-target_range) - data["Close"] >= 0)
predict_data["Target"] = np.where(predict_data["Target"] == True, 1, 0)
predict_data.dropna(inplace=True)
predict_data.reset_index(inplace=True,drop=True)
predict_data = predict_data[0:len(predict_data)-target_range]

In [7]:
predict_data

,Unix,Date,Volume USD,Variation,RSI,MACD,MACD_H,-DM,+DM,ADX14,Variation-1,Vol-1,RSI-1,MACD-1,MACD_H-1,Variation-2,Vol-2,RSI-2,MACD-2,MACD_H-2,Variation-3,Vol-3,RSI-3,MACD-3,MACD_H-3,Variation-4,Vol-4,RSI-4,MACD-4,MACD_H-4,Target
0,1483229100,2017-01-01 00:05:00,-0.348169,-0.028336,0.910591,0.004110,0.006831,-0.288606,-0.163178,-0.652332,-0.011849,-0.348678,0.835669,0.003932,0.004996,-0.000855,-0.348678,0.828292,0.003738,0.003484,-0.000855,-0.310186,0.828292,0.003534,0.002246,-0.006352,-0.312459,-0.291721,0.003268,0.001095,1
1,1483229160,2017-01-01 00:06:00,-0.347099,-0.000855,1.110668,0.004124,0.008586,-0.296745,-0.163178,-0.665621,-0.028336,-0.348169,0.910591,0.004110,0.006831,-0.011849,-0.348678,0.835669,0.003932,0.004996,-0.000855,-0.348678,0.828292,0.003738,0.003484,-0.000855,-0.310186,0.828292,0.003534,0.002246,1
2,1483229220,2017-01-01 00:07:00,-0.348678,-0.000855,1.110668,0.003720,0.009398,-0.296745,-0.163178,-0.721027,-0.000855,-0.347099,1.110668,0.004124,0.008586,-0.028336,-0.348169,0.910591,0.004110,0.006831,-0.011849,-0.348678,0.835669,0.003932,0.004996,-0.000855,-0.348678,0.828292,0.003738,0.003484,1
3,1483229280,2017-01-01 00:08:00,-0.348678,-0.000855,0.063198,0.003137,0.009823,-0.296745,-0.163178,-0.740746,-0.000855,-0.348678,1.110668,0.003720,0.009398,-0.000855,-0.347099,1.110668,0.004124,0.008586,-0.028336,-0.348169,0.910591,0.004110,0.006831,-0.011849,-0.348678,0.835669,0.003932,0.004996,0
4,1483229340,2017-01-01 00:09:00,-0.348678,0.000977,0.063198,0.002335,0.009628,-0.296745,-0.162861,-0.762462,-0.000855,-0.348678,0.063198,0.003137,0.009823,-0.000855,-0.348678,1.110668,0.003720,0.009398,-0.000855,-0.347099,1.110668,0.004124,0.008586,-0.028336,-0.348169,0.910591,0.004110,0.006831,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2674985,1646105760,2022-03-01 03:36:00,-0.334060,0.005844,3.244144,223.709502,252.087397,0.239332,-0.163178,1.311183,-0.031184,-0.219445,3.244845,227.492277,211.666068,0.061827,-0.327929,3.245042,227.891248,170.387152,-0.000855,-0.342577,3.245042,225.606771,130.272889,0.037152,-0.348678,3.245275,221.262756,92.739061,0
2674986,1646105820,2022-03-01 03:37:00,-0.010586,0.161464,3.244144,215.610181,288.355731,-0.296745,0.289396,0.957282,0.005844,-0.334060,3.244144,223.709502,252.087397,-0.031184,-0.219445,3.244845,227.492277,211.666068,0.061827,-0.327929,3.245042,227.891248,170.387152,-0.000855,-0.342577,3.245042,225.606771,130.272889,0
2674987,1646105880,2022-03-01 03:38:00,-0.343969,0.205143,3.243746,202.140130,315.950658,-0.296745,1.189788,0.570316,0.161464,-0.010586,3.244144,215.610181,288.355731,0.005844,-0.334060,3.244144,223.709502,252.087397,-0.031184,-0.219445,3.244845,227.492277,211.666068,0.061827,-0.327929,3.245042,227.891248,170.387152,0
2674988,1646105940,2022-03-01 03:39:00,-0.174815,-0.069151,3.243036,182.098937,328.738652,0.603408,-0.163178,0.176675,0.205143,-0.343969,3.243746,202.140130,315.950658,0.161464,-0.010586,3.244144,215.610181,288.355731,0.005844,-0.334060,3.244144,223.709502,252.087397,-0.031184,-0.219445,3.244845,227.492277,211.666068,1


In [8]:
predict_data = predict_data[[i % int(max_days) == 0 for i in range(len(predict_data))]]

In [9]:
predict_data

,Unix,Date,Volume USD,Variation,RSI,MACD,MACD_H,-DM,+DM,ADX14,Variation-1,Vol-1,RSI-1,MACD-1,MACD_H-1,Variation-2,Vol-2,RSI-2,MACD-2,MACD_H-2,Variation-3,Vol-3,RSI-3,MACD-3,MACD_H-3,Variation-4,Vol-4,RSI-4,MACD-4,MACD_H-4,Target
0,1483229100,2017-01-01 00:05:00,-0.348169,-0.028336,0.910591,0.004110,0.006831,-0.288606,-0.163178,-0.652332,-0.011849,-0.348678,0.835669,0.003932,0.004996,-0.000855,-0.348678,0.828292,0.003738,0.003484,-0.000855,-0.310186,0.828292,0.003534,0.002246,-0.006352,-0.312459,-0.291721,0.003268,0.001095,1
5,1483229400,2017-01-01 00:10:00,-0.345367,0.022965,0.987989,0.001263,0.008497,-0.296745,-0.163178,-0.786936,0.000977,-0.348678,0.063198,0.002335,0.009628,-0.000855,-0.348678,0.063198,0.003137,0.009823,-0.000855,-0.348678,1.110668,0.003720,0.009398,-0.000855,-0.347099,1.110668,0.004124,0.008586,0
10,1483229700,2017-01-01 00:15:00,-0.253466,0.184380,1.087972,-0.000942,0.007898,-0.296745,-0.152078,0.279085,-0.187728,-0.348174,0.497719,-0.001253,0.005496,0.004647,-0.343427,0.515160,-0.001438,0.003907,-0.000855,-0.332168,0.515160,-0.001577,0.002758,0.156867,-0.342565,0.935494,-0.000112,0.006078,0
15,1483230000,2017-01-01 00:20:00,-0.348678,-0.000855,0.361261,-0.003473,0.005520,-0.296745,-0.163178,1.915589,-0.000855,-0.348360,0.361261,-0.003338,0.004772,-0.000855,-0.347010,0.361261,-0.003205,0.004169,0.000979,-0.348678,0.370753,-0.003056,0.003729,-0.000855,-0.346988,0.370753,-0.002915,0.003356,1
20,1483230300,2017-01-01 00:25:00,-0.348678,-0.000855,-3.145265,-0.009047,-0.009628,-0.296745,-0.163178,3.807699,-0.000855,-0.346488,-3.145265,-0.009191,-0.008083,0.305644,-0.348521,0.172745,-0.006168,0.001522,-0.000855,-0.347836,0.204509,-0.003739,0.007635,-0.182238,-0.348678,-0.601917,-0.003607,0.006457,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2674965,1646104560,2022-03-01 03:16:00,-0.348437,0.250865,-0.929292,92.971007,-95.068523,-0.296745,0.204717,1.224321,0.021846,-0.347046,-0.687088,86.802582,-92.355156,-0.056509,-0.149318,-0.654502,80.955952,-89.334102,0.237821,-0.347994,0.562833,75.558155,-85.731197,0.073672,-0.200263,0.877212,70.501022,-81.948656,0
2674970,1646104860,2022-03-01 03:21:00,-0.279171,-0.107987,-1.079244,129.599592,-95.834622,0.223597,-0.163178,2.190553,0.001782,-0.293081,-1.310286,121.587055,-97.841178,-0.006993,-0.069097,-1.320477,113.921096,-98.530579,-0.178051,-0.035255,-1.717702,106.538418,-98.333515,-0.081232,1.516504,-1.805181,99.504651,-97.253852,1
2674975,1646105160,2022-03-01 03:26:00,-0.325924,-0.063982,-0.589668,173.396463,-57.275033,-0.296745,0.663952,3.333329,0.170208,-0.348409,0.061075,164.294713,-69.871801,0.115397,-0.314976,-0.178911,155.342810,-79.553241,-0.077400,-0.344656,-0.890129,146.525038,-86.943945,-0.009501,-0.332842,-0.933081,137.942891,-92.233857,1
2674980,1646105460,2022-03-01 03:31:00,-0.345160,-0.000855,3.245328,215.337802,58.534297,0.685339,-0.163178,3.308717,-0.058983,-0.282235,3.244907,208.215973,28.007690,-0.114683,-0.345629,3.244083,200.212905,1.257690,-0.075199,-0.320134,0.047310,191.628129,-21.679497,-0.295477,-0.031275,-0.720589,182.587927,-41.232730,0


In [7]:
#corr = predict_data.corr("pearson")
#corr[["Target"]].to_clipboard()
#corr[["Target"]]

In [10]:
from sklearn.model_selection import train_test_split

In [9]:
split_mode = "RAND"

if split_mode == "RAND":
    train_data, test_data, train_labels, test_labels = train_test_split(predict_data.drop(["Date", "Unix", "Target"],axis=1), predict_data["Target"], test_size=0.2, random_state=100)
elif split_mode == "STATIC":
    train_data = predict_data[0:int(0.66*len(predict_data))].drop(["Date", "Unix", "Target"],axis=1)
    train_labels = predict_data[0:int(0.66*len(predict_data))]["Target"]
    test_data = predict_data[int(0.66*len(predict_data)):len(predict_data)].drop(["Date", "Unix", "Target"],axis=1)
    test_labels = predict_data[int(0.66*len(predict_data)):len(predict_data)]["Target"]

In [10]:
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

In [12]:
algo = "MLP"

if algo == "MLP":
    model = MLPClassifier(hidden_layer_sizes=(100,200,100),verbose=True)
elif algo == "RF":
    model = RandomForestClassifier(n_estimators=100,verbose=False)
elif algo == "CAT":
    model = CatBoostClassifier(iterations=100,depth=12)
elif algo == "TREE":
    model = DecisionTreeClassifier(splitter="random")
model.fit(train_data,train_labels)

Iteration 1, loss = 0.65398021
Iteration 2, loss = 0.64336664
Iteration 3, loss = 0.63996147
Iteration 4, loss = 0.63833888
Iteration 5, loss = 0.63697008
Iteration 6, loss = 0.63595846
Iteration 7, loss = 0.63490123
Iteration 8, loss = 0.63386533
Iteration 9, loss = 0.63308808
Iteration 10, loss = 0.63230100
Iteration 11, loss = 0.63149277
Iteration 12, loss = 0.63073962
Iteration 13, loss = 0.62990224
Iteration 14, loss = 0.62882980
Iteration 15, loss = 0.62831250
Iteration 16, loss = 0.62773880
Iteration 17, loss = 0.62650033
Iteration 18, loss = 0.62577357
Iteration 19, loss = 0.62499218
Iteration 20, loss = 0.62419966
Iteration 21, loss = 0.62347343
Iteration 22, loss = 0.62242316
Iteration 23, loss = 0.62162843
Iteration 24, loss = 0.62097936
Iteration 25, loss = 0.61968942
Iteration 26, loss = 0.61909913
Iteration 27, loss = 0.61817302
Iteration 28, loss = 0.61718370
Iteration 29, loss = 0.61618977
Iteration 30, loss = 0.61551841
Iteration 31, loss = 0.61454742
Iteration 32, los

c:\Users\Alexis\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:699: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


MLPClassifier(hidden_layer_sizes=(100, 200, 100), verbose=True)

In [50]:
model.score(test_data,test_labels)

0.5018050541516246

In [51]:
model.score(train_data,train_labels)

1.0

In [35]:
from sklearn.metrics import confusion_matrix

In [53]:
preds = model.predict(test_data)

In [54]:
conf = confusion_matrix(test_labels,preds)
conf

array([[53, 72],
       [66, 86]], dtype=int64)

In [55]:
TN = conf[0,0] / (conf[0,0] + conf[1,0])
TN

0.44537815126050423

In [56]:
TP = conf[1,1] / (conf[1,1] + conf[0,1])
TP

0.5443037974683544

## Cross Validation

In [19]:
def cross_validation_loop(predict_data, model_name):
    scores = []
    TPs = []
    TNs = []
    for randomize in range(1,20):
        train_data, test_data, train_labels, test_labels = train_test_split(predict_data.drop(["Date", "Unix", "Target"],axis=1), predict_data["Target"], test_size=0.33, random_state=randomize)
        if model_name == "MLP":
            model = MLPClassifier(hidden_layer_sizes=(200,2000,200))
        elif model_name == "CAT":
            model = CatBoostClassifier(iterations=100,depth=12,verbose=False)
        elif model_name == "RF":
            model = RandomForestClassifier(n_estimators=100,verbose=False)
        model.fit(train_data, train_labels)
        scores.append(model.score(test_data, test_labels))
        conf = confusion_matrix(test_labels,model.predict(test_data))
        TN = conf[0,0] / (conf[0,0] + conf[1,0])
        TP = conf[1,1] / (conf[1,1] + conf[0,1])
        TPs.append(TP)
        TNs.append(TN)
        print("Score : {} || TP : {} || TN : {}".format(scores[-1],TP,TN))


    return scores,TPs,TNs

In [20]:
#scores,TPs,TNs = cross_validation_loop(predict_data,"CAT")

In [24]:
from statistics import mean

In [50]:
print("CatBoost score moyen : {} || TP moyen : {} || TN moyen : {}".format(mean(scores),mean(TPs),mean(TNs)))

score moyen : 0.5067452023560707 || TP moyen : 0.5411620348700146 || TN moyen : 0.4619500842660272


In [22]:
scores,TPs,TNs = cross_validation_loop(predict_data,"RF")

Score : 0.51985559566787 || TP : 0.5694444444444444 || TN : 0.46616541353383456
Score : 0.49458483754512633 || TP : 0.5194805194805194 || TN : 0.4634146341463415
Score : 0.5054151624548736 || TP : 0.5465116279069767 || TN : 0.4380952380952381
Score : 0.48375451263537905 || TP : 0.5136986301369864 || TN : 0.45038167938931295
Score : 0.4693140794223827 || TP : 0.4722222222222222 || TN : 0.4639175257731959
Score : 0.48736462093862815 || TP : 0.525 || TN : 0.4358974358974359
Score : 0.51985559566787 || TP : 0.5280898876404494 || TN : 0.5050505050505051
Score : 0.49458483754512633 || TP : 0.5611510791366906 || TN : 0.427536231884058
Score : 0.49458483754512633 || TP : 0.5384615384615384 || TN : 0.4380165289256198
Score : 0.4657039711191336 || TP : 0.4899328859060403 || TN : 0.4375
Score : 0.4693140794223827 || TP : 0.5205479452054794 || TN : 0.4122137404580153
Score : 0.5306859205776173 || TP : 0.5714285714285714 || TN : 0.46788990825688076
Score : 0.4584837545126354 || TP : 0.5 || TN : 0.4

In [25]:
print("Random Forest score moyen : {} || TP moyen : {} || TN moyen : {}".format(mean(scores),mean(TPs),mean(TNs)))

Random Forest score moyen : 0.5040851225536767 || TP moyen : 0.538149033546309 || TN moyen : 0.4600308310035195


In [26]:
scores,TPs,TNs = cross_validation_loop(predict_data,"MLP")

Score : 0.51985559566787 || TP : 0.5675675675675675 || TN : 0.46511627906976744
Score : 0.48014440433212996 || TP : 0.5068493150684932 || TN : 0.45038167938931295
Score : 0.5126353790613718 || TP : 0.56 || TN : 0.4566929133858268
Score : 0.5234657039711191 || TP : 0.5496688741721855 || TN : 0.49206349206349204
Score : 0.4657039711191336 || TP : 0.46308724832214765 || TN : 0.46875
Score : 0.5342960288808665 || TP : 0.5734265734265734 || TN : 0.4925373134328358
Score : 0.5270758122743683 || TP : 0.541095890410959 || TN : 0.5114503816793893
Score : 0.5054151624548736 || TP : 0.5657894736842105 || TN : 0.432
Score : 0.48014440433212996 || TP : 0.5266666666666666 || TN : 0.4251968503937008
Score : 0.5090252707581228 || TP : 0.5302013422818792 || TN : 0.484375
Score : 0.5126353790613718 || TP : 0.56 || TN : 0.4566929133858268
Score : 0.5451263537906137 || TP : 0.5933333333333334 || TN : 0.4881889763779528
Score : 0.4729241877256318 || TP : 0.5131578947368421 || TN : 0.424
Score : 0.534296028

In [27]:
print("MLP score moyen : {} || TP moyen : {} || TN moyen : {}".format(mean(scores),mean(TPs),mean(TNs)))

MLP score moyen : 0.5078852365570967 || TP moyen : 0.5443692036536304 || TN moyen : 0.4653102613033886


### Using TensorFlow

In [13]:
%load_ext tensorboard

In [14]:
!rm -rf ./logs/

'rm' n'est pas reconnu en tant que commande interne
ou externe, un programme ex�cutable ou un fichier de commandes.


In [16]:
shape = (len(predict_data.columns)-3,)
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=shape),
    tf.keras.layers.Dense(200, activation='relu'),
    tf.keras.layers.Dense(2000, activation='relu'),
    tf.keras.layers.Dense(2000, activation='relu'),
    tf.keras.layers.Dense(200, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [16]:
print(train_data.shape, test_data.shape)

(427998, 28) (107000, 28)


In [17]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=['accuracy', 'Precision', 'Recall'])

In [18]:
import datetime
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [19]:
model.fit(train_data, train_labels, epochs=10, validation_data=(test_data,test_labels),callbacks=[tensorboard_callback])

Epoch 1/30
13375/13375 [==============================] - 456s 34ms/step - loss: 0.6532 - accuracy: 0.6126 - precision: 0.6121 - recall: 0.6741 - val_loss: 0.6450 - val_accuracy: 0.6218 - val_precision: 0.6481 - val_recall: 0.5781
Epoch 2/30
13375/13375 [==============================] - 459s 34ms/step - loss: 0.6436 - accuracy: 0.6249 - precision: 0.6243 - recall: 0.6806 - val_loss: 0.6397 - val_accuracy: 0.6297 - val_precision: 0.6324 - val_recall: 0.6680
Epoch 3/30
13375/13375 [==============================] - 450s 34ms/step - loss: 0.6408 - accuracy: 0.6279 - precision: 0.6284 - recall: 0.6773 - val_loss: 0.6426 - val_accuracy: 0.6301 - val_precision: 0.6414 - val_recall: 0.6363
Epoch 4/30
13375/13375 [==============================] - 504s 38ms/step - loss: 0.6387 - accuracy: 0.6302 - precision: 0.6309 - recall: 0.6776 - val_loss: 0.6373 - val_accuracy: 0.6308 - val_precision: 0.6322 - val_recall: 0.6737
Epoch 5/30
13375/13375 [==============================] - 477s 36ms/step - l

In [30]:
time = str(datetime.date.today())
#model.save('./models/1m_200_2000_200_'+time)

INFO:tensorflow:Assets written to: ./models/1m_200_2000_200_2022-09-29\assets


In [32]:
loaded_model = tf.keras.models.load_model('./models/1m_200_2000_200_'+time)

In [33]:
test_loss, test_acc = loaded_model.evaluate(test_data,  test_labels, verbose=2)

#print('\nTest accuracy:', test_acc)

3344/3344 - 21s - loss: 3.0073 - accuracy: 0.6279 - precision: 0.6242 - recall: 0.6940 - 21s/epoch - 6ms/step


ValueError: too many values to unpack (expected 2)

In [50]:
test_labels.value_counts()

1    55000
0    52000
Name: Target, dtype: int64

In [24]:
probability_model = tf.keras.Sequential([model, 
                                         tf.keras.layers.Softmax()])



In [59]:
#preds = np.argmax(probability_model.predict(test_data),axis=1)
preds = np.where(model.predict(test_data)>0.5, 1, 0)

In [60]:
np.unique(preds, return_counts=True)

(array([0, 1]), array([45852, 61148], dtype=int64))

In [17]:
conf = confusion_matrix(test_labels,preds)
conf

NameError: name 'confusion_matrix' is not defined

In [62]:
TN = conf[0,0] / (conf[0,0] + conf[1,0])
TN

0.6329276803629067

In [63]:
TP = conf[1,1] / (conf[1,1] + conf[0,1])
TP

0.6242068424151239

# Predicting 2021 with 2019 and 2020

In [11]:
data_19_20 = predict_data.loc[(predict_data["Unix"] >= 1546300800) & (predict_data["Unix"] < 1609459200)]

In [12]:
data_19_20

,Unix,Date,Volume USD,Variation,RSI,MACD,MACD_H,-DM,+DM,ADX14,Variation-1,Vol-1,RSI-1,MACD-1,MACD_H-1,Variation-2,Vol-2,RSI-2,MACD-2,MACD_H-2,Variation-3,Vol-3,RSI-3,MACD-3,MACD_H-3,Variation-4,Vol-4,RSI-4,MACD-4,MACD_H-4,Target
1051110,1546301040,2019-01-01 00:04:00,-0.346650,-0.025324,0.053628,-0.021920,-0.000780,-0.296745,-0.115606,-0.578652,0.095597,-0.343283,0.529031,-0.015081,0.017446,-0.275665,-0.331139,-0.094008,-0.019997,0.000966,0.192724,-0.175991,0.014096,-0.016292,0.010564,499.722633,-0.169705,3.246387,4.969239,13.123672,0
1051115,1546301340,2019-01-01 00:09:00,-0.329127,-0.472447,-0.466916,-0.006785,0.022812,-0.278840,-0.163178,0.120873,-0.081105,-0.267165,-0.669165,-0.020851,-0.018919,0.062605,-0.333742,-0.531060,-0.029239,-0.037302,0.110160,-0.320416,-0.190011,-0.031282,-0.035239,0.180212,-0.307819,0.112435,-0.025670,-0.013362,1
1051120,1546301640,2019-01-01 00:14:00,-0.348678,0.196340,1.088781,-0.022031,0.052748,-0.296745,-0.032829,0.633312,-0.000855,-0.348678,1.207080,-0.011928,0.068896,-0.000855,-0.348678,1.207080,-0.003887,0.076365,-0.000855,-0.348678,2.277658,0.002447,0.077830,-0.000855,-0.283465,1.965031,0.007371,0.075277,0
1051125,1546301940,2019-01-01 00:19:00,-0.348678,0.061552,-1.231287,-0.045347,0.004413,-0.296745,-0.121949,1.768295,-0.000855,-0.348588,-0.816992,-0.040947,0.015157,-0.105950,-0.348678,-0.216243,-0.041032,0.011901,-0.000855,-0.345954,-0.216243,-0.040641,0.010553,0.137914,-0.340611,0.475876,-0.034619,0.024357,0
1051130,1546302240,2019-01-01 00:24:00,-0.348678,0.169270,-0.934083,-0.057667,-0.064931,-0.296745,-0.050907,2.025432,-0.000855,-0.346812,-1.248113,-0.056799,-0.049652,-0.000855,-0.329368,-1.248113,-0.055482,-0.036241,0.023151,-0.348678,-1.017968,-0.052918,-0.022217,-0.000855,-0.331925,-1.717607,-0.050316,-0.010898,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2064405,1609457820,2020-12-31 23:37:00,0.990831,-0.105212,-0.023388,86.099574,-68.269554,-0.296745,1.015038,0.904628,0.301960,1.431451,0.301881,80.783381,-68.663920,-0.149901,-0.174119,0.043969,75.653125,-68.488062,0.079191,1.238398,0.414700,70.795434,-67.627113,0.015672,0.077544,0.693249,66.187690,-66.277857,1
2064410,1609458120,2020-12-31 23:42:00,0.948758,0.183268,-0.072194,115.809752,-48.841098,-0.296745,1.036921,2.859886,0.173072,-0.107686,0.067966,109.640797,-55.374615,-0.084107,0.435150,0.087467,103.533215,-60.439248,0.048290,0.177242,0.042012,97.582874,-64.075435,-0.205001,0.198484,-0.005340,91.747871,-66.679603,0
2064415,1609458420,2020-12-31 23:47:00,0.321061,-0.171776,3.242862,146.566267,19.020170,1.154678,-0.163178,3.657532,0.019751,0.526366,3.242862,140.884728,0.202417,-0.403943,2.616738,-1.013652,134.707776,-16.160934,0.036709,2.772590,-1.031014,128.344306,-29.744497,0.568606,1.657480,-0.416464,122.065335,-40.388057,1
2064420,1609458720,2020-12-31 23:52:00,0.147181,-0.171237,3.243784,160.556058,149.272717,0.676115,-0.163178,1.769187,-0.092077,2.297348,3.243301,160.743352,119.913106,0.046214,-0.140457,3.243301,159.072290,91.514629,0.055845,0.048990,3.243431,155.967172,65.006292,-0.212085,-0.148241,3.242313,151.693925,40.712789,1


In [13]:
data_2021 = predict_data.loc[predict_data["Unix"] >= 1609459200]

In [14]:
data_2021

,Unix,Date,Volume USD,Variation,RSI,MACD,MACD_H,-DM,+DM,ADX14,Variation-1,Vol-1,RSI-1,MACD-1,MACD_H-1,Variation-2,Vol-2,RSI-2,MACD-2,MACD_H-2,Variation-3,Vol-3,RSI-3,MACD-3,MACD_H-3,Variation-4,Vol-4,RSI-4,MACD-4,MACD_H-4,Target
2064430,1609459380,2021-01-01 00:03:00,-0.035411,0.017070,1.437312,0.486452,1.766349,0.604494,-0.163178,-0.574995,-0.075473,0.084682,1.262523,0.572733,1.641080,0.289327,1.737225,2.389313,0.720446,1.703201,1214.045712,0.493491,3.242448,46.065993,121.782922,0.043916,2.559727,3.242448,81.658718,191.481682,0
2064435,1609459680,2021-01-01 00:08:00,-0.025056,0.022950,0.417542,-0.326229,1.055277,-0.296745,0.016964,0.051797,0.297738,-0.148565,1.022397,-0.136921,1.344475,0.041965,-0.082574,1.297912,0.025713,1.505349,0.012214,-0.196942,0.806634,0.156727,1.550490,0.278150,-0.141744,1.300388,0.340786,1.726775,0
2064440,1609459980,2021-01-01 00:13:00,1.640388,-0.146590,-0.868890,-0.946685,-0.202488,-0.296745,1.670272,0.366917,0.541444,-0.123238,-0.116417,-0.822243,0.166855,-0.067646,1.059105,-0.281343,-0.735235,0.363404,0.224667,-0.139513,0.157132,-0.591698,0.670025,0.075856,-0.010583,0.159882,-0.449689,0.911287,1
2064445,1609460280,2021-01-01 00:18:00,0.440705,-0.111944,-0.418507,-0.859043,0.170810,0.486209,-0.163178,0.387179,-0.061275,-0.287596,-0.363714,-0.821848,0.234938,0.028691,-0.183870,-0.515060,-0.774573,0.312876,-0.416947,-0.144609,-0.728193,-0.852356,0.044756,0.056217,1.563090,-0.551666,-0.886675,-0.054885,0
2064450,1609460580,2021-01-01 00:23:00,-0.024904,0.073330,-0.731635,-0.841248,0.206535,1.066236,-0.163178,0.536320,-0.184985,-0.105216,-1.236472,-0.858640,0.119269,-0.227717,0.736099,-1.454281,-0.929892,-0.092872,-0.151104,-0.082928,-1.319534,-1.019712,-0.311650,0.543953,-0.071406,-0.484481,-0.916955,0.022221,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2674965,1646104560,2022-03-01 03:16:00,-0.348437,0.250865,-0.929292,92.971007,-95.068523,-0.296745,0.204717,1.224321,0.021846,-0.347046,-0.687088,86.802582,-92.355156,-0.056509,-0.149318,-0.654502,80.955952,-89.334102,0.237821,-0.347994,0.562833,75.558155,-85.731197,0.073672,-0.200263,0.877212,70.501022,-81.948656,0
2674970,1646104860,2022-03-01 03:21:00,-0.279171,-0.107987,-1.079244,129.599592,-95.834622,0.223597,-0.163178,2.190553,0.001782,-0.293081,-1.310286,121.587055,-97.841178,-0.006993,-0.069097,-1.320477,113.921096,-98.530579,-0.178051,-0.035255,-1.717702,106.538418,-98.333515,-0.081232,1.516504,-1.805181,99.504651,-97.253852,1
2674975,1646105160,2022-03-01 03:26:00,-0.325924,-0.063982,-0.589668,173.396463,-57.275033,-0.296745,0.663952,3.333329,0.170208,-0.348409,0.061075,164.294713,-69.871801,0.115397,-0.314976,-0.178911,155.342810,-79.553241,-0.077400,-0.344656,-0.890129,146.525038,-86.943945,-0.009501,-0.332842,-0.933081,137.942891,-92.233857,1
2674980,1646105460,2022-03-01 03:31:00,-0.345160,-0.000855,3.245328,215.337802,58.534297,0.685339,-0.163178,3.308717,-0.058983,-0.282235,3.244907,208.215973,28.007690,-0.114683,-0.345629,3.244083,200.212905,1.257690,-0.075199,-0.320134,0.047310,191.628129,-21.679497,-0.295477,-0.031275,-0.720589,182.587927,-41.232730,0


In [15]:
split_mode = "STATIC"

#if split_mode == "RAND":
#    train_data, test_data, train_labels, test_labels = train_test_split(predict_data.drop(["Date", "Unix", "Target"],axis=1), predict_data["Target"], test_size=0.2, random_state=100)
if split_mode == "STATIC":
    train_data = data_19_20.drop(["Date", "Unix", "Target"],axis=1)
    train_labels = data_19_20["Target"]
    test_data = data_2021.drop(["Date", "Unix", "Target"],axis=1)
    test_labels = data_2021["Target"]

In [18]:
%load_ext tensorboard

In [34]:
shape = (len(test_data.columns),)
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=shape),
    tf.keras.layers.Dense(200, activation='relu'),
    tf.keras.layers.Dense(2000, activation='relu'),
    tf.keras.layers.Dense(2000, activation='relu'),
    tf.keras.layers.Dense(200, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [35]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=['accuracy', 'Precision', 'Recall'])

In [36]:
import datetime
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [37]:
model.fit(train_data, train_labels, epochs=10, validation_data=(test_data,test_labels),callbacks=[tensorboard_callback], batch_size=64)

Epoch 1/10
3167/3167 [==============================] - 137s 42ms/step - loss: 0.4691 - accuracy: 0.7766 - precision: 0.7791 - recall: 0.7884 - val_loss: 0.8881 - val_accuracy: 0.6909 - val_precision: 0.6569 - val_recall: 0.8179
Epoch 2/10
3167/3167 [==============================] - 126s 40ms/step - loss: 0.4247 - accuracy: 0.7971 - precision: 0.8000 - recall: 0.8061 - val_loss: 0.7697 - val_accuracy: 0.7142 - val_precision: 0.7026 - val_recall: 0.7573
Epoch 3/10
3167/3167 [==============================] - 126s 40ms/step - loss: 0.4128 - accuracy: 0.8039 - precision: 0.8038 - recall: 0.8174 - val_loss: 0.6319 - val_accuracy: 0.7059 - val_precision: 0.7559 - val_recall: 0.6208
Epoch 4/10
3167/3167 [==============================] - 126s 40ms/step - loss: 0.4060 - accuracy: 0.8080 - precision: 0.8088 - recall: 0.8197 - val_loss: 0.7031 - val_accuracy: 0.7415 - val_precision: 0.7358 - val_recall: 0.7651
Epoch 5/10
3167/3167 [==============================] - 127s 40ms/step - loss: 0.399

In [38]:
test_loss, test_acc = model.evaluate(test_data,  test_labels, verbose=2)

#print('\nTest accuracy:', test_acc)

3816/3816 - 23s - loss: 0.6122 - accuracy: 0.7459 - precision: 0.7216 - recall: 0.8129 - 23s/epoch - 6ms/step


ValueError: too many values to unpack (expected 2)

In [40]:
time = str(datetime.date.today())
#model.save('./models/1m_200_2000_200_y2019_2020'+time)

INFO:tensorflow:Assets written to: ./models/1m_200_2000_200_y2019_20202022-09-29\assets
